# Favorita Hierarchical Baselines

This notebooks runs and saves the forecasts of hierarchical statistical baseline methods.

- It reads a preprocessed Favorita dataset as defined in [datasetsforecast.favorita](https://github.com/Nixtla/datasetsforecast/blob/feat/favorita_dataset/nbs/favorita.ipynb).
- It filters the dataset by `item_id`.
- It fits base forecasts using StatsForecast's `AutoARIMA`.
- It reconciles the geographic aggregation levels using HierarchicalForecast's baselines, (10 bootstrap seeds).
- It saves the reconciled forecasts, test, and train subsets for each `item_id` into its respective folder `./data/Favorita200/item_id/`

:::{.callout-warning collapse="false"}
#### Large data memory requirements

Note that in its current implementation, this notebook requires approximately 300G of memory.
You can diminish the memomry usage operating on `Favorita200`, a 200 sample of the 4036 items of the `FavoritaComplete`
dataset.
:::

In [1]:
import os
import argparse

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# %%capture
# !pip install statsforecast
# !pip install hierarchicalforecast
# !pip install git+https://github.com/Nixtla/datasetsforecast.git

In [3]:
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive

from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, TopDown, MinTrace, ERM

from hierarchicalforecast.utils import is_strictly_hierarchical
from hierarchicalforecast.utils import HierarchicalPlot, CodeTimer
from hierarchicalforecast.evaluation import scaled_crps, msse, energy_score

from datasetsforecast.favorita import FavoritaData, FavoritaInfo

import warnings
# Avoid pandas fragmentation warning and positive definite warning
warnings.filterwarnings("ignore")

/opt/conda/envs/hierarchicalforecast/lib/python3.10/site-packages/statsforecast/core.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
from typing import Optional, Union
def _metric_protections(y: np.ndarray, y_hat: np.ndarray,
                        weights: Optional[np.ndarray]) -> None:
    if not ((weights is None) or (np.sum(weights) > 0)):
        raise Exception('Sum of `weights` cannot be 0')
    if not ((weights is None) or (weights.shape == y.shape)):
        raise Exception(
        f'Wrong weight dimension weights.shape {weights.shape}, y.shape {y.shape}')

def mse(y: np.ndarray, y_hat: np.ndarray,
        weights: Optional[np.ndarray] = None,
        axis: Optional[int] = None) -> Union[float, np.ndarray]:
    _metric_protections(y, y_hat, weights)

    delta_y = np.square(y - y_hat)
    if weights is not None:
        mse = np.average(delta_y[~np.isnan(delta_y)],
                         weights=weights[~np.isnan(delta_y)],
                         axis=axis)
    else:
        mse = np.nanmean(delta_y, axis=axis)
    return mse

def rel_mse(y, y_hat, y_train, mask=None):
    if mask is None:
       mask = np.ones_like(y)
    n_series, n_hier, horizon = y.shape

    eps = np.finfo(float).eps
    y_naive = np.repeat(y_train[:,:,[-1]], horizon, axis=2)
    norm = mse(y=y, y_hat=y_naive)
    loss = mse(y=y, y_hat=y_hat, weights=mask)
    loss = loss / (norm + eps)
    return loss

class FavoritaHierarchicalDataset(object):
    # Class with loading, processing and
    # prediction evaluation methods for hierarchical data

    available_datasets = ['Favorita200', 'Favorita500', 'FavoritaComplete']

    @staticmethod
    def _get_hierarchical_scrps(hier_idxs, Y, Yq_hat, q_to_pred):
        # We use the indexes obtained from the aggregation tags
        # to compute scaled CRPS across the hierarchy levels
        # # [n_items, n_stores, n_time, n_quants] 
        scrps_list = []
        for idxs in hier_idxs:
            y      = Y[:, idxs, :]
            yq_hat = Yq_hat[:, idxs, :, :]
            scrps  = scaled_crps(y, yq_hat, q_to_pred)
            scrps_list.append(scrps)
        return scrps_list

    @staticmethod
    def _get_hierarchical_msse(hier_idxs, Y, Y_hat, Y_train):
        # We use the indexes obtained from the aggregation tags
        # to compute scaled CRPS across the hierarchy levels         
        msse_list = []
        for idxs in hier_idxs:
            y       = Y[:, idxs, :]
            y_hat   = Y_hat[:, idxs, :]
            y_train = Y_train[:, idxs, :]
            crps    = msse(y, y_hat, y_train)
            msse_list.append(crps)
        return msse_list    
    
    @staticmethod
    def _get_hierarchical_rel_mse(hier_idxs, Y, Y_hat, Y_train):
        # We use the indexes obtained from the aggregation tags
        # to compute relative MSE across the hierarchy levels
        rel_mse_list = []
        for idxs in hier_idxs:
            y       = Y[:,idxs, :]
            y_hat   = Y_hat[:,idxs, :]
            y_train = Y_train[:,idxs, :]
            level_rel_mse = rel_mse(y, y_hat, y_train)
            rel_mse_list.append(level_rel_mse)
        return rel_mse_list

    @staticmethod
    def _sort_hier_df(Y_df, S_df):
        # NeuralForecast core, sorts unique_id lexicographically
        # deviating from S_df, this class matches S_df and Y_hat_df order.
        Y_df.unique_id = Y_df.unique_id.astype('category')
        Y_df.unique_id = Y_df.unique_id.cat.set_categories(S_df.index)
        Y_df = Y_df.sort_values(by=['unique_id', 'ds'])
        return Y_df

    @staticmethod
    def _nonzero_indexes_by_row(M):
        return [np.nonzero(M[row,:])[0] for row in range(len(M))]

    @staticmethod
    def load_item_data(item_id, dataset='Favorita200', directory='./data'):
        # Load data
        data_info = FavoritaInfo[dataset]
        Y_df, S_df, tags = FavoritaData.load(directory=directory,
                                             group=dataset)

        # Parse and augment data
        # + hack geographic hier_id to treat it as unique_id
        Y_df['ds'] = pd.to_datetime(Y_df['ds'])
        Y_df = Y_df[Y_df.item_id==item_id]
        Y_df = Y_df.rename(columns={'hier_id': 'unique_id'})
        Y_df = FavoritaHierarchicalDataset._sort_hier_df(Y_df=Y_df, S_df=S_df)

        # Obtain indexes for plots and evaluation
        hier_levels = ['Overall'] + list(tags.keys())
        hier_idxs = [np.arange(len(S_df))] +\
            [S_df.index.get_indexer(tags[level]) for level in list(tags.keys())]
        hier_linked_idxs = FavoritaHierarchicalDataset._nonzero_indexes_by_row(S_df.values.T)

        # MinT along other methods require a positive definite covariance matrix
        # for the residuals, when dealing with 0s as residuals the methods break
        # data is augmented with minimal normal noise to avoid this error.
        Y_df['y'] = Y_df['y'] + np.random.normal(loc=0.0, scale=0.01, size=len(Y_df))

        # Final output
        data = dict(Y_df=Y_df, S_df=S_df, tags=tags,
                    # Hierarchical idxs
                    hier_idxs=hier_idxs,
                    hier_levels=hier_levels,
                    hier_linked_idxs=hier_linked_idxs,
                    # Dataset Properties
                    horizon=data_info.horizon,
                    freq=data_info.freq,
                    seasonality=data_info.seasonality)
        return data
    
    @staticmethod
    def load_process_data(dataset='Favorita200', directory='./data'):
        # Load data
        data_info = FavoritaInfo[dataset]
        Y_df, S_df, tags = FavoritaData.load(directory=directory,
                                             group=dataset)

        # Parse and augment data
        # + hack geographic hier_id to treat it as unique_id
        Y_df['ds'] = pd.to_datetime(Y_df['ds'])
        #Y_df = Y_df[Y_df.item_id==item_id]
        Y_df = Y_df.rename(columns={'hier_id': 'unique_id'})
        Y_df = FavoritaHierarchicalDataset._sort_hier_df(Y_df=Y_df, S_df=S_df)

        # Obtain indexes for plots and evaluation
        hier_levels = ['Overall'] + list(tags.keys())
        hier_idxs = [np.arange(len(S_df))] +\
            [S_df.index.get_indexer(tags[level]) for level in list(tags.keys())]
        hier_linked_idxs = FavoritaHierarchicalDataset._nonzero_indexes_by_row(S_df.values.T)

        # MinT along other methods require a positive definite covariance matrix
        # for the residuals, when dealing with 0s as residuals the methods break
        # data is augmented with minimal normal noise to avoid this error.
        Y_df['y'] = Y_df['y'] + np.random.normal(loc=0.0, scale=0.01, size=len(Y_df))

        # Final output
        data = dict(Y_df=Y_df, S_df=S_df, tags=tags,
                    # Hierarchical idxs
                    hier_idxs=hier_idxs,
                    hier_levels=hier_levels,
                    hier_linked_idxs=hier_linked_idxs,
                    # Dataset Properties
                    horizon=data_info.horizon,
                    freq=data_info.freq,
                    seasonality=data_info.seasonality)
        return data
    
    

In [5]:
data = FavoritaHierarchicalDataset.load_item_data(item_id=112830, directory = './data/favorita', dataset='Favorita200')

In [6]:
def run_baselines(item_id, data, intervals_method='bootstrap', dataset='Favorita200', verbose=False, seed=0):
    with CodeTimer('Read and Parse data   ', verbose):
        Y_df_item = data['Y_df'][data['Y_df'].item_id ==item_id]
        Y_df = Y_df_item[["unique_id", 'ds', 'y']]
        S_df, tags = data['S_df'], data['tags']
        horizon = data['horizon']
        seasonality = data['seasonality']
        freq = data['freq']

        # Train/Test Splits
        Y_test_df  = Y_df.groupby('unique_id').tail(horizon)
        Y_train_df = Y_df.drop(Y_test_df.index)
        Y_test_df  = Y_test_df.set_index('unique_id')
        Y_train_df = Y_train_df.set_index('unique_id')

        dataset_str = f'{dataset} item_id={item_id}, h={horizon} '
        dataset_str += f'n_series={len(S_df)}, n_bottom={len(S_df.columns)} \n'
        dataset_str += f'test ds=[{min(Y_test_df.ds), max(Y_test_df.ds)}] '

    with CodeTimer('Fit/Predict Model	  ', verbose):
        # Read to avoid unnecesary AutoARIMA computation
        item_path = f'./data/{dataset}/{item_id}'
        os.makedirs(item_path, exist_ok=True)
        yhat_file = f'{item_path}/Y_hat.csv'
        yfitted_file = f'{item_path}/Y_fitted.csv'
        yrec_file = f'{item_path}/{intervals_method}_rec.csv'

        if os.path.exists(yhat_file):
            Y_hat_df = pd.read_csv(yhat_file)
            Y_fitted_df = pd.read_csv(yfitted_file)

        else:
            if not os.path.exists(f'./data/{dataset}'):
                os.makedirs(f'./data/{dataset}')			
            fcst = StatsForecast(
                df=Y_train_df, 
                models=[AutoARIMA(season_length=seasonality)],
                fallback_model=[Naive()],
                freq=freq, 
                n_jobs=-1
            )
            Y_hat_df = fcst.forecast(h=horizon, fitted=True, level=LEVEL)
            Y_fitted_df = fcst.forecast_fitted_values()

            Y_hat_df = Y_hat_df.reset_index()
            Y_fitted_df = Y_fitted_df.reset_index()
            Y_hat_df.to_csv(yhat_file, index=False)
            Y_fitted_df.to_csv(yfitted_file, index=False)

        Y_hat_df = Y_hat_df.set_index('unique_id')
        Y_fitted_df = Y_fitted_df.set_index('unique_id')

    with CodeTimer('Reconcile Predictions ', verbose):
        if is_strictly_hierarchical(S=S_df.values.astype(np.float32), 
            tags={key: S_df.index.get_indexer(val) for key, val in tags.items()}):
            reconcilers = [
#                 BottomUp(),
#                 TopDown(method='average_proportions'),
#                 TopDown(method='proportion_averages'),
                 MinTrace(method='ols'),
#                 MinTrace(method='mint_shrink', mint_shr_ridge=1e-6),
                #ERM(method='reg_bu', lambda_reg=100) # Extremely inneficient
                #ERM(method='closed')
            ]
        else:
            reconcilers = [
#                 BottomUp(),
                 MinTrace(method='ols'),
#                 MinTrace(method='mint_shrink', mint_shr_ridge=1e-6),
                #ERM(method='reg_bu', lambda_reg=100) # Extremely inneficient
                #ERM(method='closed')
            ]
        
        hrec = HierarchicalReconciliation(reconcilers=reconcilers)
        Y_rec_df = hrec.bootstrap_reconcile(Y_hat_df=Y_hat_df,
                                            Y_df=Y_fitted_df,
                                            S_df=S_df, tags=tags,
                                            level=LEVEL,
                                            intervals_method=intervals_method,
                                            num_samples=10, num_seeds=10)

        # Matching Y_test/Y_rec/S index ordering
        Y_test_df = Y_test_df.reset_index()
        Y_test_df.unique_id = Y_test_df.unique_id.astype('category')
        Y_test_df.unique_id = Y_test_df.unique_id.cat.set_categories(S_df.index)
        Y_test_df = Y_test_df.sort_values(by=['unique_id', 'ds'])

        Y_rec_df = Y_rec_df.reset_index()
        Y_rec_df.unique_id = Y_rec_df.unique_id.astype('category')
        Y_rec_df.unique_id = Y_rec_df.unique_id.cat.set_categories(S_df.index)
        Y_rec_df = Y_rec_df.sort_values(by=['seed', 'unique_id', 'ds'])

        #Y_rec_df.to_csv(yrec_file, index=False)

        
        # Parsing model level columns
        flat_cols = list(hrec.level_names.keys())
        for model in hrec.level_names:
            flat_cols += hrec.level_names[model]
        for model in hrec.sample_names:
            flat_cols += hrec.sample_names[model]
        y_rec  = Y_rec_df[flat_cols]
        model_columns = y_rec.columns
        model = list(hrec.level_names.keys())[0]
        col_idx = model_columns.get_indexer(hrec.level_names[model])
        col_idx_mean = model_columns.get_loc(model)
        n_series = len(S_df)
        n_seeds = len(Y_rec_df.seed.unique())
        y_rec  = y_rec.values.reshape(n_seeds, n_series, horizon, len(model_columns))
        y_rec_model = y_rec[:,:,:,col_idx]
        y_rec_mean = y_rec[:,:,:,col_idx_mean]
        y_test = Y_test_df['y'].values.reshape(n_series, horizon)
        y_train = Y_train_df['y'].values.reshape(n_series, -1)
        
        
#         # [n_items, n_samples, horizon, n_hier] 0,1,2,3
#         # -> [n_items, n_hier, horizon, n_samples] 0,3,2,1
#         samples = np.concatenate(samples_list, axis=0)
#         samples = np.transpose(samples, (0,3,2,1))
#         #col_idx = model_columns.get_loc(model)
#         Y_hat  = np.mean(samples, axis=3)
#         Yq_hat = np.quantile(samples, q=QUANTILES, axis=3)
#         Yq_hat = np.transpose(Yq_hat, (1,2,3,0))

#         Y_test = data['Y_hier'][:,:,-data['horizon']:]
#         Y_train = data['Y_hier'][:,:,:-data['horizon']]
        
        
        return y_rec_model,y_rec_mean, y_test, y_train
    
  

In [7]:
# Parse execution parameters
verbose = False
intervals_method = 'bootstrap'
dataset = 'FavoritaComplete'

assert intervals_method in ['bootstrap', 'normality', 'permbu'], \
    "Select `--intervals_method` from ['bootstrap', 'normality', 'permbu']"

available_datasets = ['Favorita200', 'Favorita500', 'FavoritaComplete']
assert dataset in available_datasets, \
    "Select `--dataset` from ['Favorita200', 'Favorita500', 'FavoritaComplete']"

LEVEL = np.arange(0, 100, 2)
qs = [[50-lv/2, 50+lv/2] for lv in LEVEL]
QUANTILES = np.sort(np.concatenate(qs)/100)

# Run experiments
Y_all_df, S_df, tags = FavoritaData.load(directory='./data/favorita/', group=dataset)
items = Y_all_df.item_id.unique()

print('\n')
print(f'{intervals_method.upper()} {dataset} statistical baselines evaluation \n')
Y_rec_list = []
Y_test_list = []
Y_train_list = []
Y_rec_mean_list = []
directory = './data/favorita'
data = FavoritaHierarchicalDataset.load_process_data(dataset=dataset, directory = directory)
import sys
#with open('./log3.txt', 'w') as sys.stdout:
for count,item_id in enumerate(items):
    Y_rec_curr,Y_rec_mean_curr, Y_test_curr, Y_train_curr = run_baselines(item_id = item_id, data =data, dataset=dataset,intervals_method=intervals_method,verbose=verbose)
    Y_rec_list.append(Y_rec_curr)
    Y_rec_mean_list.append(Y_rec_mean_curr)
    Y_test_list.append(Y_test_curr)
    Y_train_list.append(Y_train_curr)
        #print(f'{count} ')
    



BOOTSTRAP FavoritaComplete statistical baselines evaluation 



In [8]:
Y_rec = np.array(Y_rec_list)
Y_rec_mean = np.array(Y_rec_mean_list)

Y_test = np.array(Y_test_list)
Y_train = np.array(Y_train_list)


In [9]:
Y_rec_mean.shape

(4036, 10, 93, 34)

In [11]:
results_df = pd.DataFrame(dict(level=['Overall']+list(data['tags'].keys())))


hier_idxs = data['hier_idxs']
LEVEL = np.arange(0, 100, 2)
qs = [[50-lv/2, 50+lv/2] for lv in LEVEL]
QUANTILES = np.sort(np.concatenate(qs)/100)

seed_model_crps = []
seed_model_relmse = []
for seed_i in range(10):
    Y_rec_i = np.maximum(Y_rec[:,seed_i,:,:,:],0)
    Y_hat = np.maximum(Y_rec_mean[:,seed_i,:,:],0)
    #Y_hat = Y_rec_i.mean(axis=-1)
    curr_crps = FavoritaHierarchicalDataset._get_hierarchical_scrps(hier_idxs, Y_test, Y_rec_i, q_to_pred=QUANTILES)
    curr_relmse = FavoritaHierarchicalDataset._get_hierarchical_rel_mse(hier_idxs, Y_test, Y_hat, Y_train = Y_train)
    seed_model_crps.append(curr_crps)
    seed_model_relmse.append(curr_relmse)

seed_model_crps = np.array(seed_model_crps)
seed_model_relmse = np.array(seed_model_relmse)

levels_crps = []
levels_relmse = []
for level_j in range(results_df.shape[0]):
    curr_level_crps = f'{np.mean(seed_model_crps[:,level_j],axis=0):.4f}±{(1.96 * np.std(seed_model_crps[:,level_j],axis=0)):.4f}'
    curr_level_relmse = f'{np.mean(seed_model_relmse[:,level_j],axis=0):.4f}±{(1.96 * np.std(seed_model_relmse[:,level_j],axis=0)):.4f}'

    levels_crps.append(curr_level_crps)
    levels_relmse.append(curr_level_relmse)

results_df['scrps'] = levels_crps
results_df['rel_mse'] = levels_relmse

results_df.to_csv(f'./data/{intervals_method}_metrics_minT_ols.csv', index=False)


In [12]:
results_df

,level,scrps,rel_mse
0,Overall,0.3968±0.0007,1.1270±0.0000
1,Country,0.2652±0.0005,1.0833±0.0000
2,Country/State,0.3782±0.0006,1.1164±0.0000
3,Country/State/City,0.4028±0.0008,1.1341±0.0000
4,Country/State/City/Store,0.5410±0.0010,1.3157±0.0000
